In [86]:
#initialize dataframe
import pandas as pd

file = "Resources/purchase_data.csv"
p_df = pd.read_csv(file)

In [87]:
# Get number of unique players, make dataframe from scratch
pd.DataFrame([p_df["SN"].nunique()], columns = ["Total Players"])

,Total Players
0,576


In [88]:
#create dataframe inplace with formatting included for a summary of purchases
pd.DataFrame([[p_df["Item Name"].nunique(), "${price:.2f}".format(price = p_df["Price"].mean()), \
               p_df["Purchase ID"].count(),"${price:.2f}".format(price = p_df["Price"].sum())]], \
               columns = ["Number of Unique", "Items Average Price", "Number of Purchases", "Total Revenue"])

,Number of Unique,Items Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


In [89]:
#reduce df by usernames, get gender counts, convert to dataframe. Afterwards append new column of percentage with nice formatting

gender_df = p_df.drop_duplicates(subset=["SN"])["Gender"].value_counts().to_frame()
gender_df["Percentage of Players"] = (gender_df["Gender"]/gender_df["Gender"].sum()).apply(lambda x : x * 100).map("{:.2f}%".format)
gender_df

,Gender,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [92]:
#This function will be used for gender and age analysis
def analysis(x):
        d = []
        a = x['SN'].count()
        b = x['Price'].sum()
        c = x['SN'].nunique()
        d.append(a)
        d.append("${x:.2f}".format(x = b))
        d.append("${x:.2f}".format(x = b/a))
        d.append("${x:.2f}".format(x = b/c))
        return pd.Series(d, index=['Purchase Count', 'Total Purchase Value', 'Average Purchase Price', 'Avg Total Purchase per Person'])

In [93]:
p_df[["Gender","SN", "Price"]].groupby("Gender").apply(analysis)


,Purchase Count,Total Purchase Value,Average Purchase Price,Avg Total Purchase per Person
Gender,,,,
Female,113,$361.94,$3.20,$4.47
Male,652,$1967.64,$3.02,$4.07
Other / Non-Disclosed,15,$50.19,$3.35,$4.56


In [94]:
#create copy of original dataframe with new age range row
bins = [-float('inf')] + [5*i - 1 for i in range(2,9)] + [float('inf')]
groupnames = ["<10"] + [str(i + 1)+'-'+str(i + 5) for i in bins[1:-2]] + [">40"]
pages_df = p_df.copy()
pages_df["Age Ranges"] = pd.cut(pages_df["Age"], bins, labels = groupnames, include_lowest = False)

In [96]:
#count unique players of age ranges given then calculate and fornat percentages
agecount_df = pages_df[["Age Ranges","SN"]].drop_duplicates(subset=["SN"]).groupby("Age Ranges").count().rename(columns = {"SN":"Total Count"})
agecount_df["Percentage of Players"] = (agecount_df["Total Count"]/agecount_df["Total Count"].sum()).apply(lambda x : x * 100).map("{:.2f}%".format)
agecount_df.index.rename(None, inplace = True)
agecount_df

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
>40,12,2.08%


In [97]:
pages_df[["Age Ranges", "SN", "Price"]].groupby("Age Ranges").apply(analysis)

,Purchase Count,Total Purchase Value,Average Purchase Price,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$77.13,$3.35,$4.54
10-14,28,$82.78,$2.96,$3.76
15-19,136,$412.89,$3.04,$3.86
20-24,365,$1114.06,$3.05,$4.32
25-29,101,$293.00,$2.90,$3.81
30-34,73,$214.00,$2.93,$4.12
35-39,41,$147.67,$3.60,$4.76
>40,13,$38.24,$2.94,$3.19


In [106]:
topspenders = pd.merge(p_df[["SN","Price"]].groupby("SN").sum().sort_values("Price", ascending = False).head(5), \
                       p_df[["SN","Price"]].groupby("SN").count(), \
                       on = "SN")

topspenders["Average Purchase Value"] = topspenders["Price_x"]/topspenders["Price_y"]
topspenders = topspenders[["Price_y","Average Purchase Value","Price_x"]] \
                         .rename(columns = {"Price_x" : "Average Purchase Price", "Price_y" : "Purchase Count"})

for i in topspenders.columns[1:]:
    topspenders[i] = topspenders[i].map("${:.2f}".format)

topspenders

,Purchase Count,Average Purchase Value,Average Purchase Price
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [111]:
def itemsummary(x):
        d = []
        a = x['Price'].count()
        b = x['Price'].sum()
        d.append(a)
        d.append("${x:.2f}".format(x = b/a))
        d.append("${x:.2f}".format(x = b))
        return pd.Series(d, index=['Purchase Count', 'Item Price', 'Total Purchase Value'])


t = p_df[['Item ID', 'Item Name','Price']].groupby(['Item ID','Item Name']).apply(itemsummary)
t.sort_values(by = "Purchase Count", ascending = False).head(10)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
75,Brutality Ivory Warmace,8,$2.42,$19.36
103,Singed Scalpel,8,$4.35,$34.80
34,Retribution Axe,8,$2.22,$17.76


In [104]:
pd.merge(t, (t["Total Purchase Value"].map(lambda x: float(x[1:]))).rename("float"), on = ['Item ID', 'Item Name']) \
    .sort_values(by = "float", ascending = False) \
    .drop("float", axis = 1).head(10)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
59,"Lightning, Etcher of the King",8,$4.23,$33.84
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
78,"Glimmer, Ender of the Moon",7,$4.40,$30.80
72,Winter's Bite,8,$3.77,$30.16


In [130]:
bins = [0.99,1.99,2.99,3.99,4.99]
labels = ["1-2", "2-3", "3-4", "4-5"]

price_df = p_df.copy()
price_df["Price Range"] = pd.cut(price_df["Price"], bins, labels = labels, include_lowest = False)
price_analysis = price_df[["Price","Price Range"]].groupby("Price Range").count()
price_analysis.rename(columns = {"Price" : "Purchases at Price"}, inplace = True)

In [135]:
price_analysis["Items at Price"] = price_df.drop_duplicates("Item Name")[["Price","Price Range"]].groupby("Price Range").count()
price_analysis["Purchases per Price"] = price_analysis["Purchases at Price"]/price_analysis["Items at Price"]
price_analysis

,Purchases at Price,Items at Price,Purchases per Price
Price Range,,,
1-2,200,45,4.444444
2-3,152,39,3.897436
3-4,213,47,4.531915
4-5,215,48,4.479167
